In [ ]:
import polars as pl
import seaborn as sb
import gtfs_delay_analysis as da

In [ ]:
import importlib
importlib.reload(da)

In [ ]:
stops = pl.read_csv(
    '/home/chrlz/dox/dl/ETS_Bus_Schedule_GTFS_Data_Feed_-_Stops_20240216.csv')

In [ ]:
aggregated = da.load_aggregate_data()

So we know that all trip id is unique for a single day, no need to worry about overlaps

Average delay in a stop every 3 minutes 10 recordings of a bus

AM: 7am-9am
PM: 4pm-7pm
OFF: 5am-7am, 9am-4-pm, 7pm-10pm


In [ ]:
# 1. Map delay average over week
mean_stop = aggregated.pipe(da.agg_group, 'stopid')

# 2. Over week by day
# TODO: Filter by PEAK, OFF, and DAY


def pivot_day(df: pl.DataFrame):
    def get_day(d: str):
        if 'day' not in d:
            return d
        col, _, d = d.split('_')
        days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        return f'{days[int(d)-1]}_{col[:3]}'
    return (
        df.select('day', 'hour', 'avgdelay', 'stddelay')
        .pivot(values=['avgdelay', 'stddelay'], index='hour', columns='day')
        .rename(get_day)
    )


def by_day(df: pl.DataFrame):
    by_day_all = df.pipe(da.agg_group, 'day')
    by_day_peak = (
        df
        .filter(pl.col('period') != 'OFF')
        .pipe(da.agg_group, 'day')
    )
    by_day_off = (
        df
        .filter(pl.col('period') == 'OFF')
        .pipe(da.agg_group, 'day')
    )

    def get(df: pl.DataFrame, prefix: str):
        return df.select(
            'day',
            pl.col('avgdelay').alias(f'{prefix}_avg'),
            pl.col('stddelay').alias(f'{prefix}_std'),
        )
    g_day = (
        by_day_all.pipe(get, 'all')
        .join(by_day_peak.pipe(get, 'peak'), on='day', how='left')
        .join(by_day_off.pipe(get, 'off'), on='day', how='left')
    )

    # 3. Over day by hour
    by_hour = df.pipe(da.agg_group, 'day', 'hour')
    return g_day, by_hour, by_hour.pipe(pivot_day)


network_by_day, by_hour, by_hour_pivot = by_day(aggregated)

# TODO: Select route
# 4. Time series graph of route by hour by day


highest_delay_stops = (
    mean_stop
    .group_by('stopid')
    .agg(pl.col('avgdelay').max())
    .sort('avgdelay', descending=True)
    .select('stopid')
    .head(100)
    .unique()
    .join(aggregated, on='stopid')
)
routes_on_highest_delay_stops = (
    highest_delay_stops
    .select('routeid')
    .unique()
    .join(aggregated, on='routeid')
)
agg_routes_on_highest_delay = (
    routes_on_highest_delay_stops
    .pipe(da.agg_group, 'routeid')
    .drop('lastupdate')
)


def plot_cols(df: pl.DataFrame, key: str):
    tidy = df.melt(id_vars=key).sort(key)
    xticks = tidy[key]
    ax = sb.barplot(
        data=tidy,
        x=key,
        y='value',
        hue='variable',
        width=1,
        order=xticks,
    )
    return tidy, ax


def plot_by_route(df: pl.DataFrame):
    tidy, ax = plot_cols(df, 'routeid')
    xticks_unique = tidy['routeid'].unique()
    ax.set_xticks(ax.get_xticks(), xticks_unique, rotation=90)


# 5. Mapping delay propagations within a route
# 6. TODO

def plot_stop_and_route(df: pl.DataFrame, stop: str, route: str):
    days = pl.DataFrame({
        "day": [*range(1, 6)],
        "letter": ['M', 'T', 'W', 'R', 'F'],
    }, schema={"day": pl.Int8, 'letter': pl.Categorical})
    by_day_by_hour_for_stop = (
        da.select_stop_and_route(df, stop, route)
        .join(days, on='day')
        .drop('day')
        .rename({'letter': 'day'})
    )

    ax = sb.barplot(by_day_by_hour_for_stop, x='hour', y='avgdelay', hue='day')
    ax.set_title(f'Average delay for route {route} stop {stop}')
    return by_day_by_hour_for_stop


# select_stop(aggregated, "2260")

# sb.barplot(by_day_by_hour_for_stop, x='hour', y='maxdelay')


# highest_delay_stops.select('stopid').unique()

# plot_stop_and_route(aggregated, "2260", "637")
# da.select_stop(aggregated, "2260")

In [ ]:
def filter_by_stop_and_route(agg: pl.DataFrame, stopid: str, routeid: str):
    g_day, g_hour, g_hour_pivot = (
        agg
        .filter(
            (pl.col('stopid') == stopid)
            & (pl.col('routeid') == routeid)
        )
        .pipe(by_day)
    )

    g_hour.drop('lastupdate').write_csv(f'{stopid}-{routeid}-by-hour.csv')
    g_hour_pivot.write_csv(f'{stopid}-{routeid}-by-hour-pivot.csv')
    g_day.write_csv(f'{stopid}-{routeid}-by-day.csv')
    return g_day, g_hour

In [ ]:
_, bd = aggregated.pipe(filter_by_stop_and_route, '2260', '004')
_, bd = aggregated.pipe(filter_by_stop_and_route, '2260', '637')
m637 = aggregated.filter(
    pl.col('routeid').__eq__('637').and_(
        pl.col('stopid').__eq__('2260')
    )
)['meandelay'].mean()

m004 = aggregated.filter(
    pl.col('routeid').__eq__('004').and_(
        pl.col('stopid').__eq__('2260')
    )
)['meandelay'].mean()
m637, m004

In [ ]:
raw_dfs = da.load_raw_data()

In [ ]:
raw_dfs.filter(
    pl.col('routeid').__eq__('637').and_(
        pl.col('stopid').__eq__('2260')
    )
)

In [ ]:
aggregated.filter((pl.col('stopid') == "2260")
                  & (pl.col('routeid') == "637")
                  )

In [ ]:
*_, network_pivot = aggregated.pipe(by_day)

In [ ]:
aggregated.filter(pl.col('stopid') == '1899').group_by('routeid').agg(
    pl.col('meandelay').mean().alias('avgdelay'),
    pl.col('maxdelay').max().alias('maxdelay'),
    pl.col('count').sum()
).write_csv('1899-by-route.csv')

In [ ]:
mean_stop.pipe(da.add_coords, stops).drop(
    'lastupdate').write_csv('mean-stop.csv')

In [ ]:
network_by_day.drop('lastupdate').write_csv('network-by-day.csv')
by_hour.drop('lastupdate').write_csv('network-by-day-by-hour.csv')
# by_hour.drop('lastupdate')

In [ ]:

# plot_cols(df, 'day') and None

# plot_cols(by_hour.select('hour', 'day', 'avgdelay'), 'hour') and None

# by_hour

to_plot = by_hour.select('day', 'hour', 'avgdelay')
ax = sb.lineplot(to_plot, x='hour', y='avgdelay', hue='day')
ax.set_xticks([*range(5, 23)]) and None

In [ ]:
(
    mean_stop.drop('lastupdate')
    .pipe(da.add_coords, stops)
    .write_csv('mean-stop.csv')
)

In [ ]:
len(mean_stop) - len(mean_stop.filter(pl.col('numtrips') > 200))
# len(mean_stop)

In [ ]:
by_hour

Some exploration of the `trips.json` file which DOES contain coordinates of the
trip

From initial observation of the data, each trip may have a unique path, even for
the same route (e.g. shorter route for off-peak hours, my bus route home was
like that at one point)

Other assumptions:

- There is only one type of geometry line: MultiLineString
- The actual coordinates are wrapped inside another JSON array


In [ ]:
trips = da.load_trips_df()

In [ ]:
pl.Config.set_fmt_str_lengths(1000)
trips.sort('route_id')

In [ ]:
import geopolars as gp
gp.GeoDataFrame(mean_stop.pipe(da.add_coords, stops)).plot()

How many counts should we deem useful for visualization of data?


In [ ]:
pl.Config.set_fmt_table_cell_list_len(100)
aggregated['routeid']
# 1_250_916

In [ ]:
# plot_stop_and_route(aggregated, '1899', '560')
# plot_stop_and_route(aggregated, '1899', '413')
# plot_stop_and_route(aggregated, '5281', '002')
# plot_stop_and_route(aggregated, '5281', '004')

aggregated.filter(
    (pl.col('stopid') == '2260')
    # & (pl.col('routeid') == '904')
).group_by('routeid').agg(
    pl.col('count').sum(),
    pl.col('meandelay').sum(),
)
# 25529677

da.select_stop(aggregated, '2260')

# plot_stop_and_route(aggregated, '2260', '004')
plot_stop_and_route(aggregated, '2260', '637')

In [ ]:
shapes = da.trips.load_parsed_shapes_df()

In [ ]:
trips = da.trips.load_trips_without_shapes_df()
shapes = da.trips.load_str_shapes_df()

In [ ]:
25536739

In [ ]:
trips.filter(pl.col('trip_id') == 25536739).join(
    shapes,
    on='shape_id'
).select('route_id', 'shape_id').unique().join(
    shapes, on='shape_id'
).write_csv(f'004-{25536739}-shapes.csv')

In [ ]:
# Find trip with highest delay on Tuesday for route 560
aggregated.filter(
    (pl.col('routeid') == "560")
    & (pl.col('day') == 2)
).group_by('id').agg(
    pl.col('meandelay').max()
).sort('meandelay', descending=True)

In [ ]:
# Find all points for that trip
def filter_by_trip(id: int, day: int):
    df = aggregated.filter(
        (pl.col('id') == id) &
        (pl.col('day') == day)
    ).pipe(da.add_coords, stops).sort('meandelay').drop('delay')
    df.write_csv(f'{id}-{day}-stops.csv')
    return df


# filter_by_trip(25527827, 2)
# filter_by_trip(25527777, 2)
# filter_by_trip(25527796, 2)

In [ ]:
most_delayed_trips = [*aggregated.filter(pl.col('routeid') == "637").group_by('id', 'date', 'day').agg(
    pl.col('meandelay').max(),
).sort('meandelay', descending=True).head(3).select('id', 'day').iter_rows()]

In [ ]:
most_delayed_trips

In [ ]:
# tr = trips.filter(pl.col('shape_id') == "004-168-West").join(
#     shapes, on='shape_id'
# ).explode('geometry_line').with_columns(
#     pl.col('geometry_line').struct.field('lon'),
#     pl.col('geometry_line').struct.field('lat')
# )

pl.Config.set_fmt_str_lengths(100)

(
    aggregated.join(
        trips.filter(pl.col('shape_id') ==
                     "004-168-West").select(id='trip_id'),
        on='id'
    )
    .with_columns(pl.col('stopid').cast(pl.Utf8))
    .join(
        stops,
        left_on="stopid",
        right_on="stop_id"
    )
    .with_columns(
        pl.col('geometry_point')
        .str.strip_prefix("POINT (")
        .str.strip_suffix(")")
        .str.split(" ")
        .list.eval(pl.element().cast(pl.Float64))
        .list.to_struct(fields=["lon", "lat"])
    ).with_columns(
        pl.col('geometry_point').struct.field('lon'),
        pl.col('geometry_point').struct.field('lat')
    )
    .join(tr, left_on='id', right_on="trip_id")
    .with_columns(
        pl.col('lon')
        .sub(pl.col('lon_right'))
        .abs()
        .add(pl.col('lat').sub(pl.col('lat_right')).abs())
        .alias('manhattan')
    )
    .filter(pl.col('manhattan') == pl.col('manhattan').min().over('stopid'))
    # .sort('id')
    # .filter()
)

# tr
# tr['geometry_line'].struct.fields

In [ ]:
aggregated.join(
    trips.filter(pl.col('shape_id') == "004-168-West").select(id='trip_id'),
    on='id'
).group_by('stopid').all()

In [ ]:
rev_1 = (
    aggregated.join(
        trips.select('trip_headsign', 'route_id', id='trip_id'),
        on='id',
    )
    .group_by('route_id', 'trip_headsign').agg(
        pl.col('meandelay').mean(),
        pl.col('count').sum(),
        pl.col('id').n_unique()
    )
    .sort('meandelay', descending=True)
    # .write_csv('avg-delay-by-trip-headsign.csv')
)
"""
1. Join `trips` to `aggregated` to new dataframe, only adding `trip_headsign` column. 

 *THEN* GroupBy(`trip_headsign`) + Aggregate(average the delay). 
Drop all columns except the Heading, Aggregated Average Delay

-> Sort by descending meandelay
"""

max_trip_headsign = rev_1.filter(pl.col('id') > 30)[0]

In [ ]:
rev_2_1 = (
    aggregated.join(
        trips.select('trip_headsign', 'route_id', id='trip_id'),
        on='id',
    )
    .filter(
        (pl.col('routeid') == max_trip_headsign['route_id'][0]) &
        (pl.col('trip_headsign') == max_trip_headsign['trip_headsign'][0])
    )
    .group_by('id')
    .agg(
        pl.col('route_id').first(),
        pl.col('trip_headsign').first(),
        pl.col('meandelay').mean(),
        pl.col('count').sum(),
    )
    .sort('meandelay', 'route_id', descending=True)
    # .write_csv('avg-by-trip-id-508-Meadows.csv')
)
"""
2.1 Based on a given **Heading**: Select all associated rows, GroupBy(TripId) + 
Aggregate(Average the delay). 
-> Sort by descending meandelay. Select highest meandelay of the few. 
(tripid is the identifier)
"""

In [ ]:
def join_stops(df: pl.DataFrame, stops: pl.DataFrame):
    return (
        df.with_columns(pl.col('stopid').cast(pl.Utf8))
        .join(stops, left_on='stopid', right_on='stop_id')
    )


def make_sequence(
    df: pl.DataFrame,
    trip_id: int | None = None,
    shape_id: str | None = None,
):
    trips = da.trips.load_trips_without_shapes_df()
    shapes = da.trips.load_parsed_shapes_df()
    by_stop = df.group_by('stopid').agg(
        pl.col('meandelay').mean(),
        pl.col('stop_lon').first(),
        pl.col('stop_lat').first(),
        pl.col('routeid').first().cast(pl.Utf8),
    )
    pred = (
        pl.col('trip_id').eq(trip_id) if trip_id
        else pl.col('shape_id').eq(shape_id) if shape_id
        else True)
    trip_points = (
        trips
        .filter(pred)
        .join(shapes, on='shape_id')
        .unique('shape_id')
        .explode('geometry_line')
        .unique('geometry_line', keep='first')
        .with_row_index()
    )
    return (
        trip_points.join(
            by_stop,
            left_on='route_id',
            right_on='routeid',
        )
        .with_columns(
            pl.col('geometry_line').struct.field(
                'lon').sub(pl.col('stop_lon')),
            pl.col('geometry_line').struct.field(
                'lat').sub(pl.col('stop_lat')),
        )
        # Get euclidean distance
        .with_columns(
            pl.col('lon').pow(2).add(
                pl.col('lat').pow(2)).sqrt().alias('euclidean')
        )
        # Get the minimum euclidean distance for a stop
        .filter(pl.col('euclidean').eq(pl.col('euclidean').min().over('stopid')))
        # Re-create index
        .sort('index')
        .drop('index')
        .with_row_index()
        .select([
            'index',
            'route_id',
            'trip_headsign',
            'stopid',
            'meandelay',
            'stop_lon',
            'stop_lat',
        ])
    )


selected_trip = 25536592
agg_points = (
    aggregated
    .filter(pl.col('id').eq(selected_trip))
    .pipe(join_stops, stops)
)
rev_2_2 = make_sequence(agg_points, trip_id=selected_trip)
"""
2.2 Based on the selected `trip_headsign` **AND** `trip_id`: Select all the 
associated rows, join co-ordinates to StopId, join sequence #
"""
# rev_2_2.write_csv(f'rev-2.2-508-meadows-{selected_trip}-stops.csv')


selected_shape = "508-1-East"
agg_points = (
    aggregated
    .join((trips
           .filter(pl.col('shape_id').eq(selected_shape))
           .select('trip_id', 'shape_id')
           ),
          left_on='id',
          right_on='trip_id'
          )
    .pipe(join_stops, stops)
)
rev_2_3 = make_sequence(agg_points, shape_id=selected_shape)
"""
2.3 Based on the selected **Heading**: Select all the associated rows, 
GroupBy(Sequence #) + Aggregate(Average the delay) 

-> Output df: **Sequence # | AverageAverageDelay**
"""
# rev_2_3.write_csv(f'rev-2.3-{selected_shape}-stops.csv')

In [ ]:
# trips.filter(pl.col('trip_headsign').eq('508 Meadows'))['shape_id'].value_counts()

In [ ]:
# trips['shape_id'].unique()

In [ ]:
trips.write_csv('trips-new.csv')
shapes.write_csv('shapes-new.csv')

In [ ]:
aggregated.filter(pl.col('id') == 25536770).sort('lastupdate')

In [ ]:
for trip_id, day in most_delayed_trips:
    filter_by_trip(trip_id, day)

In [ ]:
raw_dfs = da.load_raw_data()

In [ ]:
# mean_stop.filter(pl.col('count') > 20).sort('avgdelay', descending=True)

# pl.Config

more_than_one_route = (
    aggregated
    .group_by('stopid')
    .agg(pl.col('routeid').unique())
    .with_columns(pl.col('routeid').list.len())
    .filter(pl.col('routeid') > 1)
    .select('stopid')
)

# aggregated.join(more_than_one_route, on='stopid')
mean_stop.join(more_than_one_route, on='stopid').sort(
    'avgdelay', descending=True)

(
    aggregated.filter(pl.col('stopid') == "2260")
    ['routeid'].unique()
)

In [ ]:
trips.filter(pl.col('route_id') == "004")